In [15]:
%load_ext cython

import numpy as np

from isocedar.datasets import MalanchevDataset
from sklearn.ensemble import IsolationForest

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


* Dataset is an o-by-f array, where o is objects and f is features.
* Labels is a one-dimentional o array with three types of labels:
 * -1 for anomalies,
 * 0 for unknowns,
 * 1 for regular data.

In [8]:
class RandomPineForest:
    def __init__(self, trees=100, subsamples=256, depth=None, seed=0):
        self.subsamples = subsamples
        self.trees = trees
        self.depth = depth
        
        self.seedseq = np.random.SeedSequence(seed)
        self.rng = np.random.default_rng(seed)
        
        self.estimators = []
        self.n = 0

    def fit(self, data):
        n = data.shape[0]
        self.n = n
        self.subsamples = self.subsamples if n > self.subsamples else n
        
        self.depth = self.depth or int(np.ceil(np.log2(self.subsamples)))

        self.estimators = [None] * self.trees
        seeds = self.seedseq.spawn(self.trees)
        for i in range(self.trees):
            subs = self.rng.choice(n, self.subsamples)
            gen = RandomPineGenerator(data[subs, :], self.depth, seeds[i])
            self.estimators[i] = gen.pine
        
        return self

    def mean_paths(self, data):
        means = np.zeros(data.shape[0])
        for ti in range(self.trees):
            means += self.estimators[ti].paths(data)
        
        means /= self.trees
        return means

    def scores(self, data):
        means = self.mean_paths(data)
        return - 2 ** (-means / average_path_length(self.subsamples))


class RandomPine:
    def __init__(self, features, selectors, values):
        self.features = features
        self.len = selectors.shape[0]
        
        # Two complementary arrays.
        # Selectors select feature to branch on.
        self.selectors = selectors
        # Values either set the deciding feature value or set the closing path length
        self.values = values

    def _get_one_path(self, key):
        i = 1
        while 2 * i < self.selectors.shape[0]:
            f = self.selectors[i]
            if f < 0:
                break
            
            if key[f] <= self.values[i]:
                i = 2 * i
            else:
                i = 2 * i + 1
        
        return self.values[i]
    
    def paths(self, x):
        n = x.shape[0]
        paths = np.empty(n)
        for i in range(n):
            paths[i] = self._get_one_path(x[i, :])
        
        return paths


class RandomPineGenerator:
    def __init__(self, sample, depth, seed=0):
        self.depth = depth
        self.features = sample.shape[1]
        self.length = 1 << (depth + 1)
        self.rng = np.random.default_rng(seed)
        self.selectors = np.full(self.length, -1, dtype=np.int32)
        self.values = np.full(self.length, 0, dtype=np.float64)
        
        self._populate(1, sample)
        self.pine = RandomPine(self.features, self.selectors, self.values)
    
    def _populate(self, i, sample):
        if sample.shape[0] == 1:
            self.values[i] = np.floor(np.log2(i))
            return
        
        if self.length <= 2 * i:
            self.values[i] = np.floor(np.log2(i)) + \
                average_path_length(sample.shape[0])
            return
        
        selector = self.rng.integers(self.features)
        self.selectors[i] = selector

        minval = np.min(sample[:, selector])
        maxval = np.max(sample[:, selector])
        if minval == maxval:
            self.values[i] = np.floor(np.log2(i)) + \
                average_path_length(sample.shape[0])
            return

        value = self.rng.uniform(minval, maxval)
        self.values[i] = value

        self._populate(2 * i, sample[sample[:, selector] <= value])
        self._populate(2 * i + 1, sample[sample[:, selector] > value])


def average_path_length(n):
    if n <= 1:
        return 0
    elif n == 2:
        return 1
    else:
        return 2.0 * (np.log(n - 1.0) + np.euler_gamma) - 2.0 * (n - 1.0) / n

In [82]:
%%cython

import numpy as np
cimport numpy as np


def pine_sample_scores(selectors, values, data):
    scores = np.zeros(data.shape[0])
    _sample_scores(selectors, values, data, scores)
    return scores


cdef void _sample_scores(np.ndarray[np.int_t, ndim=2] selectors,
                         np.ndarray[np.double_t, ndim=2] values,
                         np.ndarray[np.double_t, ndim=2] data,
                         np.ndarray[np.double_t] scores):
    cdef Py_ssize_t i = 0
    
    for i in range(scores.shape[0]):
        scores[i] = data[i, 0]

In [83]:
import numpy as np


class PineEvaluator:
    def __init__(self, pine_forest):
        pines = pine_forest.estimators
        self.trees = len(pines)
        if self.trees < 1:
            raise ValueError('a forest without trees?')

        self.leafs = pines[0].selectors.shape[0]

        self.selectors = np.empty((self.trees, self.leafs), dtype=int)
        self.values = np.empty((self.trees, self.leafs), dtype=float)
        
        #self._populate(self, pines)
    
    def _populate(self, pines):
        for it in range(len(pines)):
            if pines[it].selectors.shape[0] != self.leafs:
                raise ValueError('trees must be of equal depth to use evaluator')

            self.selectors[it, :] = pines[it].selectors
            self.values[it, :] = pines[it].values
    
    def sample_score(self, x):
        return pine_sample_scores(self.selectors, self.values, x)

In [84]:
pe = PineEvaluator(forest)
pe.sample_score(dataset.data)

array([0.31848084, 0.02048676, 0.40663512, ..., 1.1062101 , 1.08159646,
       1.09534329])

In [85]:
dataset.data[:, 0]

array([0.31848084, 0.02048676, 0.40663512, ..., 1.1062101 , 1.08159646,
       1.09534329])

In [9]:
dataset = MalanchevDataset(inliers=2**16, outliers=4)

In [10]:
%%time
forest = RandomPineForest().fit(dataset.data)

CPU times: user 478 ms, sys: 2.92 ms, total: 481 ms
Wall time: 481 ms


In [11]:
%%time
scores = forest.scores(dataset.data)

CPU times: user 2min 11s, sys: 8.71 ms, total: 2min 11s
Wall time: 2min 11s


In [16]:
%%time
isoforest = IsolationForest().fit(dataset.data)

CPU times: user 365 ms, sys: 3 ms, total: 368 ms
Wall time: 367 ms


In [17]:
%%time
isoscores = isoforest.score_samples(dataset.data)

CPU times: user 1.43 s, sys: 10 ms, total: 1.44 s
Wall time: 1.44 s
